In [2]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_countries,check_doi_glutton,normalize,aplatir,remove_too_long
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [3]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [4]:
pd.Series([list(pd.Series([x[i].get('wg') for i in range(len(x))]).drop_duplicates()) for x in list(df_ipcc.drop_duplicates(subset=['doi']).dropna(subset=['doi']).wg_chap)]).value_counts()

[2]                   21803
[3]                   12324
[1]                   10478
[2_cross]              3491
[2, 2_cross]           1531
[1, 2]                 1459
[2, 3]                 1226
[1, 2, 2_cross]         251
[1, 3]                  189
[1, 2_cross]            176
[1, 2, 3]               172
[2, 2_cross, 3]          94
[2_cross, 3]             34
[1, 2, 2_cross, 3]       24
[1, 2_cross, 3]           6
Name: count, dtype: int64

In [5]:
min([int(x) for x in list(df_ipcc.year_OA.drop_duplicates().dropna())])

1686

In [6]:
df_ipcc.test_glutton.value_counts()

test_glutton
True    74098
Name: count, dtype: int64

In [7]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [8]:
df_ipcc.columns

Index(['year', 'url', 'doi', 'title', 'author', 'freq', 'wg', 'chap',
       'countries', 'concepts', 'sdg', 'topics', 'is_OA_available', 'title_OA',
       'year_OA', 'authors_name', 'rors', 'institutions_names',
       'locations_names', 'ipcc', 'test_glutton'],
      dtype='object')

In [9]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0],'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [10]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [11]:
len(df_ipcc_group.doi.drop_duplicates())==len(df_ipcc_group.doi)

True

In [12]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [13]:
df_ipbes.columns

Index(['doi', 'title', 'date', 'creators', 'tags', 'countries', 'concepts',
       'sdg', 'topics', 'is_OA_available', 'year_OA', 'authors_name', 'rors',
       'institutions_names', 'locations_names', 'year'],
      dtype='object')

In [14]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [15]:
df_ipbes.countries.value_counts()

countries
[US]            1164
[None]           933
[GB]             338
[AU]             223
[CA]             214
                ... 
[GB, DE, NP]       1
[BE, BJ, NG]       1
[CU, US]           1
[ID, BO]           1
[CL, US, CA]       1
Name: count, Length: 1583, dtype: int64

In [16]:
df_ipbes[df_ipbes.doi=='10.1016/j.cosust.2016.12.006'].countries[84]

['GB',
 'DE',
 'MX',
 'JP',
 'GH',
 'US',
 'TR',
 'SD',
 'KR',
 'SE',
 'FR',
 'DK',
 'HU',
 'AR',
 'SA',
 'ES']

In [17]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0],'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0]})
df_ipbes_group=df_ipbes_group.reset_index()

In [18]:
df_ipbes_group[df_ipbes_group.doi=='10.1016/j.cosust.2016.12.006'].countries[909]

['GB',
 'DE',
 'MX',
 'JP',
 'GH',
 'US',
 'TR',
 'SD',
 'KR',
 'SE',
 'FR',
 'DK',
 'HU',
 'AR',
 'SA',
 'ES']

In [19]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [20]:
len(df_ipbes_group.doi.drop_duplicates())==len(df_ipbes_group.doi)

True

In [21]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year_x', 'rors_x', 'authors_name_x', 'institutions_names_x',
       'ipcc', 'countries_x', 'ipbes']].rename(columns={'year_x':'year', 'rors_x':'rors','authors_name_x':'authors_name','institutions_names_x':'institutions_names', 'countries_x':'countries'})

In [22]:
len(df_ipcc_ipbes.doi.drop_duplicates())==len(df_ipcc_ipbes.doi)

True

In [44]:
update_bso_publications()

2024-03-27 17:04:15,809 | code_utils.BSO | DEBUG | update bso publications files


DEBUG:code_utils.BSO:update bso publications files
c:\Users\haallat\Documents\teds\code_utils\BSO.py:34: DtypeWarning: Columns (17,18,21,22,23,24,25,27,28,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bso = pd.read_csv('https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-publications-latest.csv.gz', sep=';')


2024-03-27 17:07:55,059 | code_utils.BSO | DEBUG | writing 1813337 dois info from bso publications


DEBUG:code_utils.BSO:writing 1813337 dois info from bso publications


In [23]:
bso=pd.DataFrame(get_bso_publications())

In [24]:
df_bso=bso.transpose().reset_index()

In [25]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [26]:
df_bso

,doi,rors,bso_local_affiliations
0,10.47421/sllmoo23_51-66,[04zmssz18],[130008121]
1,10.26434/chemrxiv.14095699.v2,[04zmssz18],[130008121]
2,10.26434/chemrxiv.13713328.v1,[04zmssz18],[130008121]
3,10.1101/2021.03.01.433388,[04zmssz18],[130008121]
4,10.20944/preprints202107.0318.v1,[04zmssz18],[130008121]
...,...,...,...
1813332,10.1038/srep30379,"[0495fxg12, 04w6kn183]","[775684897, 784314064, gip-be, sante_pub_inser..."
1813333,10.1038/s41567-020-0989-3,"[055ss7a31, 013cjyk83, 02en5vm52]","[199411684F, 130026149, 303623, 130023385, pcc..."
1813334,10.1101/147884,[],[]
1813335,10.1162/qss_a_00146,[],[]


In [27]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [28]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [29]:
del df_ipcc_ipbes_bso['rors_y']

In [30]:
df_ipcc_ipbes_bso.columns

Index(['doi', 'year', 'rors', 'authors_name', 'institutions_names', 'ipcc',
       'countries', 'ipbes', 'bso_local_affiliations'],
      dtype='object')

In [31]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [32]:
df_iso

,name,code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
238,Wallis and Futuna,WF
239,Western Sahara,EH
240,Yemen,YE
241,Zambia,ZM


In [33]:
dict_iso={}
for i,row in df_iso.iterrows():
    dict_iso[row['code']]=row['name']
dict_iso['CW']='Curaçao'
dict_iso['RS']='Serbia'
dict_iso['ME']='Montenegro'
dict_iso['XK']='Kosovo'
dict_iso['SS']='South Soudan'
dict_iso['MF']='Saint Martin'

In [34]:
df_ipcc_ipbes_bso.loc[:,'countries']=df_ipcc_ipbes_bso.loc[:,'countries'].apply(lambda x: [dict_iso[y] for y in x] if (x!=None)&(x!=[])&(isinstance(x,float)==False) else None)

In [ ]:
df_brevet_doi=pd.read_json('brevet_doi.json')

In [58]:
df_brevet_doi

,doi,docdb_family_id,appln_publn_number,appln_auth
0,10.1038/s41477-019-0461-5,80461940,2022171736,WO
1,10.1126/science.aad1329,68610090,2021063948,WO
2,10.1126/science.aad1329,68610090,4038388,EP
3,10.1126/science.aad1329,82358379,2022268841,WO
4,10.1016/j.enpol.2011.09.036,54185996,3030841,FR
...,...,...,...,...
95,10.1126/science.aad1329,61074267,3505177,EP
96,10.1126/science.aad1329,61074267,2019129753,WO
97,10.1002/ejoc.201200111,48795535,2821791,EP
98,10.1002/ejoc.201200111,48795535,2015001117,WO


In [112]:
df_ipcc_ipbes_bso_brevet=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [114]:
len(df_ipcc_ipbes_bso_brevet.doi.drop_duplicates())==len(df_ipcc_ipbes_bso_brevet.doi)

False

In [118]:
df_ipcc_ipbes_bso_brevet.doi.value_counts()[:10]

doi
10.1126/science.aad1329                   15
10.1038/nature05414                        8
10.1016/j.cell.2014.03.011                 5
10.1007/s11356-014-3470-y                  5
10.1126/science.1223490                    4
10.1186/s40779-020-00240-0                 4
10.5194/acp-14-13023-2014                  3
10.1146/annurev.phyto.40.120301.093927     3
10.1016/j.agrformet.2013.04.012            3
10.1016/j.renene.2011.03.005               3
Name: count, dtype: int64

In [35]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

54821 data in cached openalex data
6608 data in cached openalex data


In [36]:
df_cache

,doi,OpenAlex_data
0,10.1098/rstl.1686.0026,"[{'id': 'https://openalex.org/W4244279015', 'd..."
1,10.1098/rstl.1735.0014,"[{'id': 'https://openalex.org/W4252425265', 'd..."
2,10.1098/rstl.1861.0001,"[{'id': 'https://openalex.org/W3091998362', 'd..."
3,10.1017/s0080456800017592,"[{'id': 'https://openalex.org/W2314134964', 'd..."
4,10.2307/2992177,"[{'id': 'https://openalex.org/W256209766', 'do..."
...,...,...
60245,10.9774/gleaf.978-1-907643-06-4_5,"[{'id': 'https://openalex.org/W3124428460', 'd..."
60246,10.9783/9780812295948-002,"[{'id': 'https://openalex.org/W3137011924', 'd..."
60247,10.2172/2283221,"[{'id': 'https://openalex.org/W4391458189', 'd..."
60248,10.1016/b978-0-12-812362-1.00002-3,"[{'id': 'https://openalex.org/W3107290679', 'd..."


In [37]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [38]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [39]:
dict_doi_bso=[]

In [40]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [41]:
len(dict_doi_bso)

58240

In [42]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [43]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [44]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', lines=True)

In [45]:
dd.ipbes.value_counts()

ipbes
[{'chapter': '3'}]                                                                                                                  1277
[{'chapter': '2.1_drivers'}]                                                                                                        1002
[{'chapter': '6'}]                                                                                                                   926
[{'chapter': '4'}]                                                                                                                   773
[{'chapter': '2.2_nature'}]                                                                                                          552
                                                                                                                                    ... 
[{'chapter': '2.2_nature'}, {'chapter': '2.3_ncp'}, {'chapter': '4'}]                                                                  1
[{'chapter': '1'}, {'chapter': '3'}

In [ ]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [ ]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

In [46]:
current_dict = dd.to_dict(orient='records')

In [47]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [48]:
df_elastic_input=pd.DataFrame(current_dict)

In [49]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [50]:
del df_elastic_input['openalex_abstract_inverted_index']
del df_elastic_input['openalex_primary_topic']

In [51]:
df_elastic_input=df_elastic_input[['doi', 'year', 'rors', 'ipcc','authors_name', 'institutions_names',
       'countries', 'ipbes', 'openalex_authorships','openalex_topics']]

In [52]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [53]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [ ]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
len(df)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'